In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import requests
import shutil
import glob
import time
import datetime
import os
import re
import csv
import pandas as pd
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
import pywordseg as seg  # https://pypi.org/project/pywordseg/
import nltk
import contractions
from nltk.corpus import stopwords 
#nltk.download('vader_lexicon')
#nltk.download('popular')
#%pip install xlsxwriter
# from TCSP import read_stopwords_list
# %pip install tcsp

In [ ]:
# https://realpython.com/python-nltk-sentiment-analysis/
# https://machinelearningmastery.com/clean-text-machine-learning-python/
# https://www.guru99.com/pos-tagging-chunking-nltk.html
# https://www.nltk.org/api/nltk.tokenize.html
# https://github.com/bryanchw/Traditional-Chinese-Stopwords-and-Punctuations-Library

In [ ]:
# script_dir = os.path.dirname(r"C:\Users\ericdao\Downloads\\") #to specify another directory to output to

script_dir = os.getcwd()
out_folder = str(datetime.date.today()) + "_output"
abs_path = os.path.join(script_dir, out_folder)
# or can specifiy a specific folder if working on a previous existing output 
# abs_path = "2022-04-29_output" 

if os.path.exists(abs_path):
    shutil.rmtree(abs_path)
os.mkdir(abs_path)


# set up webscraping driver
chromedriver = './chromedriver_100' #mac doesn't requite extenstion in name, whereas, windows does. Driver download page: https://chromedriver.chromium.org/downloads
options = webdriver.ChromeOptions()
options.add_experimental_option('excludeSwitches', ['enable-logging'])  #prevents logging of some unecessary driver related logs
driver = webdriver.Chrome(chromedriver, options=options) 

# url = "https://google.com"
# driver.get(url)

In [2]:
#grab the US song titles

page = requests.get("https://www.billboard.com/charts/year-end/2021/hot-100-songs")
soup = BeautifulSoup(page.content, "html.parser")

job_elements = soup.find_all("h3", class_="c-title") 
song_titles = []
for song_title in job_elements:
    song_titles.append(song_title.text.strip())

song_titles_100 = song_titles[:100] #verify that that this list contains all the song titles


In [15]:
# use song titles to search the lyrics. Will need to manually fetch and input any song lyrics that weren't able to be automatically web scraped

file_name = "us_song_lyrics_2021"
lyrics = []
with open((abs_path + "//" + file_name +'.csv'), 'w', newline='') as f:
    write = csv.writer(f)
    for song in song_titles_100:
        try:
            matched_elements = driver.get("https://www.google.com/search?q=" + song + " lyrics")
            time.sleep(1)
            x = driver.find_element_by_css_selector('div[jsname="WbKHeb"]')
            lyrics.append(x.text)
            write.writerow([x.text])
        except Exception as e:
            print("error | ",song, " | ", e) # there will be some lyrics that won't automatically show up in google's first search result. These will need to be manually found and inserted into the us_song_lyrics csv file
            pass
    f.close()    
    print(len(lyrics) + " songs' lyrics retrieved")

error Drivers License Message: no such element: Unable to locate element: {"method":"css selector","selector":"div[jsname="WbKHeb"]"}
  (Session info: chrome=100.0.4896.127)

error Beautiful Mistakes Message: no such element: Unable to locate element: {"method":"css selector","selector":"div[jsname="WbKHeb"]"}
  (Session info: chrome=100.0.4896.127)

error Kings & Queens Message: no such element: Unable to locate element: {"method":"css selector","selector":"div[jsname="WbKHeb"]"}
  (Session info: chrome=100.0.4896.127)



In [37]:
# webscaper for tw song lyrics. Some song indexes will give an error like "frame disconnected", I think this relates to maybe an error casued by trying to scrape something when the page hasn't loaded completely yet maybe. I played around with putting wait intervals between different web scraping steps, and that seemed to help, but I need to spend more time to figure out exactly what causes that error in this script section

driver = webdriver.Chrome(chromedriver, options=options) 

file_name = "tw_song_lyrics_2020"
songs = []
with open((abs_path + "//" + file_name +'.csv'), 'w', newline='', encoding="UTF-8") as f:
    write = csv.writer(f)
    for i in range(0,202,2):
        try:
            driver.get("https://kma.kkbox.com/charts/yearly/newrelease?cate=297&lang=tc&terr=tw&year=2020#")
            time.sleep(5)
            page = driver.find_elements_by_class_name("charts-list-cover")
            time.sleep(2)
            new_page = page[i].click()
            time.sleep(3)
            page = requests.get(driver.current_url)
            time.sleep(3)

            soup = BeautifulSoup(page.content, "html.parser")
            time.sleep(2)
            job_elements = soup.find("div", class_="lyrics")
            time.sleep(3)
            raw_text = job_elements.getText().splitlines()
            while True:
                for a in range(len(raw_text)): 
                    author_line_1 = raw_text[a].find('：') #-1 will be ruturned for find() if no match is found
                    author_line_2 = raw_text[a].find(':')
                    if (author_line_1 == -1 and author_line_2 == -1) and raw_text[a] != '': #this will skip those first few lines with the "：" symbol and also the blank lines
                        break #once the first line is found with the ： and isn't blank, the loop will break and now a is the index of the start of the actual song to extract
                break
            text = raw_text[a:]
            text = str("\n".join(text))
            text = text.replace("\n", " ")
            songs.append(text)
            write.writerow([text])
            print(i)
        except Exception as e:
            print("error",i, e)
            pass
    f.close()  
    print(len(songs))

0


In [146]:
# manually fetch any tw song indexes that weren't sucessfully pulled and append to existing csv

driver = webdriver.Chrome(chromedriver, options=options) 

file_name = "tw_song_lyrics_2021"
songs = []
with open((abs_path + "//" + file_name +'.csv'), 'a', newline='', encoding="UTF-8") as f: #adjust directory path
    write = csv.writer(f)
    for i in ([22]):
        try:
            driver.get("https://kma.kkbox.com/charts/yearly/newrelease?cate=297&lang=tc&terr=tw&year=2021#") #adjust year
            time.sleep(5)
            page = driver.find_elements_by_class_name("charts-list-cover")
            time.sleep(2)
            new_page = page[i].click()
            time.sleep(3)
            page = requests.get(driver.current_url)
            time.sleep(3)

            soup = BeautifulSoup(page.content, "html.parser")
            time.sleep(2)
            job_elements = soup.find("div", class_="lyrics")
            time.sleep(3)
            raw_text = job_elements.getText().splitlines()
            while True:
                for a in range(len(raw_text)): 
                    author_line_1 = raw_text[a].find('：') #-1 will be ruturned for find() if no match is found
                    author_line_2 = raw_text[a].find(':')
                    if (author_line_1 == -1 and author_line_2 == -1) and raw_text[a] != '': #this will skip those first few lines with the "：" symbol and also the blank lines
                        break #once the first line is found with the ： and isn't blank, the loop will break and now a is the index of the start of the actual song to extract
                break
            text = raw_text[a:]
            text = str("\n".join(text))
            text = text.replace("\n", " ")
            songs.append(text)
            write.writerow([text])
            print(i)
        except Exception as e:
            print("error",i, e)
            pass
    f.close()  
    print(len(songs))

22
1


In [50]:
# insert manually obtained lyrics into csv

lyrics = """我們走過了多少的傷害
然後經歷了又一次失敗
I know i’ll be alright
Everything will be alright alright

我們走過了多少的傷害
然後經歷了又一次失敗
I know i’ll be alright
Everything will be alright alright be alright

Ya 放輕鬆 全部重新來過
繼續 向前走 把煩惱丟走
不再留戀 別再回頭
We moving on run n run like a circle
認真的去感受頻率
希望停止那些回憶
回憶都只是過去 讓他過去 過去

我們走過了多少的傷害（傷害）
然後經歷了又一次失敗
I know i’ll be alright（i know i’ll be alright）
Everything will be alright alright

我們走過了多少的傷害（傷害）
然後經歷了又一次失敗
I know i’ll be alright（i know i’ll be alright）
Everything will be alright alright be alright
Be alright i i’ll be alright
Be alright i i’ll be alright（i know i’ll be alright）
Be alright i i’ll be alright
Be alright i i’ll be alright
（everything will be alright alright be alright）

獨自一人走在大街
孤獨一人聽著音樂
不敢再犯錯我一步一步的走
曾經想要解脫 但又從頭來過
不再回頭 保持初心向前走
所有煩惱拋腦後
認真的過每天生活

I’ll go insane if i stay
All this way with no change all day
Undo the pain
Freshen up
Get back up
Moving on go on

我們走過了多少的傷害（傷害）
然後經歷了又一次失敗
I know i’ll be alright（i know i’ll be alright）
Everything will be alright alright（be alright）
我們走過了多少的傷害（傷害）
然後經歷了又一次失敗
I know i’ll be alright（i know i’ll be alright）
Everything will be alright alright be alright

Be alright i i’ll be alright
Be alright i i’ll be alright（i know i’ll be alright）
Be alright i i’ll be alright
Be alright i i’ll be alright
（everything will be alright alright be alright）

Be alright i i’ll be alright
Be alright i i’ll be alright（i know i’ll be alright）
Be alright i i’ll be alright
Be alright i i’ll be alright
Everything will be alright alright be alright"""

raw_text = lyrics.splitlines()

with open((abs_path + "//" + file_name +'.csv'), 'a', newline='', encoding="UTF-8") as f:
    write = csv.writer(f)
    while True:
        for a in range(len(raw_text)): 
            author_line_1 = raw_text[a].find('：') #-1 will be ruturned for find() if no match is found
            author_line_2 = raw_text[a].find(':')
            if (author_line_1 == -1 and author_line_2 == -1) and raw_text[a] != '': #this will skip those first few lines with the "：" symbol and also the blank lines
                break #once the first line is found with the ： and isn't blank, the loop will break and now a is the index of the start of the actual song to extract
        break
    text = raw_text[a:]
    text = str("\n".join(text))
    text = text.replace("\n", " ")
    songs.append(text)
    write.writerow([text])          

In [42]:
# output each individual word count for tw words

file_name = "tw_song_lyrics_2020"


# flattten song lyric list to prepare for segmentation
df = pd.read_csv(abs_path + "//" + file_name + ".csv", encoding="UTF-8", header=None)
songs_to_segment = df.values.tolist()
flattened_song_list = ' '.join([data for song in songs_to_segment for data in song])


# declare the segmentor. embedding="w2v" will use the baseline model, "elmo" is supposely more accurate, but slower
segmentor = seg.Wordseg(batch_size=64, device="cpu", embedding='w2v', elmo_use_cuda=False, mode="TW")

# input is a flattened string of all the sentences.
lyric_segmentation_result = segmentor.cut([flattened_song_list])[0]
# test = segmentor.cut(["今天天氣真好啊! 潮水退了就知道，誰沒穿褲子。"])


# remove punctuation characters
punctuation = pd.read_csv("tw_punctuation.csv", encoding="UTF-8", header=None)
punctuation = punctuation[0].values.tolist()
lyric_segmentation = [word.strip().lower() for word in lyric_segmentation_result if not word.strip().lower() in punctuation and not "'" in word.strip().lower() and not "," in word.strip().lower() and not "’" in word.strip().lower()]
# lyric_segmentation = [word.strip().lower() for word in lyric_segmentation if not "'" in word and not "," in word]
# lyric_segmentation = [word.lower() for word in lyric_segmentation not word.__contains__("'")]


# remove stop words
stop_words = pd.read_csv("tw_stopwords.csv", encoding="UTF-8", header=None)
stop_words = stop_words[0].values.tolist()
lyric_rm_stopwords_segmentation = [word.strip().lower() for word in lyric_segmentation_result if not word.strip().lower() in stop_words and not "'" in word.strip().lower() and not "," in word.strip().lower() and not "’" in word.strip().lower()]
# lyric_rm_stopwords_segmentation = [word.lower() for word in lyric_rm_stopwords_segmentation if word not in "'"]

# calculate each word's word count
def countOccurrence(a):
  k = {}
  for j in a:
    if j in k:
      k[j] +=1
    else:
      k[j] =1
  return k

lyric_segmentation_count = countOccurrence(lyric_segmentation)
lyric_rm_stopwords_segmentation_count = countOccurrence(lyric_rm_stopwords_segmentation)

# sort word count
sorted_keys = sorted(lyric_segmentation_count, key=lyric_segmentation_count.get, reverse=True)
sorted_lyric_segmentation_count = {}
for w in sorted_keys:
    sorted_lyric_segmentation_count[w] = lyric_segmentation_count[w]

sorted_keys_rm_stopwords = sorted(lyric_rm_stopwords_segmentation_count, key=lyric_rm_stopwords_segmentation_count.get, reverse=True)
sorted_rm_stopwords_lyric_segmentation_count = {}
for w in sorted_keys_rm_stopwords:
    sorted_rm_stopwords_lyric_segmentation_count[w] = lyric_rm_stopwords_segmentation_count[w]

# convert dictionary to dataframe
sorted_lyric_segmentation_count_df = pd.DataFrame(list(sorted_lyric_segmentation_count.items()))
sorted_lyric_segmentation_count_df.columns = ["character", "count"]

sorted_lyric_rm_stopwords_segmentation_count_df = pd.DataFrame(list(sorted_rm_stopwords_lyric_segmentation_count.items()))
sorted_lyric_rm_stopwords_segmentation_count_df.columns = ["character", "count"]

# export
sorted_lyric_segmentation_count_df.to_csv (abs_path + "//" + file_name + "_word_count.csv", index = False, header=True, encoding="UTF-8")
sorted_lyric_rm_stopwords_segmentation_count_df.to_csv (abs_path + "//" + file_name + "_rm_stopwords_word_count.csv", index = False, header=True, encoding="UTF-8")

In [38]:
# output each individual word count for us songs

file_name = "us_song_lyrics_2020"

# flattten song lyric list to prepare for segmentation
df = pd.read_csv(abs_path + "//" + file_name + ".csv", encoding="UTF-8", header=None)
songs_to_segment = df.values.tolist()
flattened_song_list_1 = ' '.join([data.lower() for song in songs_to_segment for data in song])

flattened_song_list_2 = str("\n".join([flattened_song_list_1]))
flattened_song_list_2 = flattened_song_list_2.replace("\n", " ")

# seperate contractions into two seperate words
flattened_song_list_3 = contractions.fix(flattened_song_list_2) 

# input is a list of string sentences that need to be flatenned.
lyric_segmentation = nltk.word_tokenize(flattened_song_list_3)

# remove punctuation characters (will also remove any words that include a puncuation character)
lyric_segmentation = [word for word in lyric_segmentation if word.isalpha()] 

# remove stop words
stop_words = set(nltk.corpus.stopwords.words('english'))
lyric_rm_stopwords_segmentation = [w for w in lyric_segmentation if not w in stop_words]


# segment words
lyric_pos = nltk.pos_tag(lyric_segmentation)
lyric_rm_stopwords_pos = nltk.pos_tag(lyric_rm_stopwords_segmentation)


# make it easier to categorize words in post by categorizing based on part of speech. Update- it turns out nltk's part of speech categorizer isn't the most accurate. For example the number of times that love is referenced in the feelings category is much less than without part of speech categorization. More investigation is needed to determine if it's worth using.
def feelings(pos_tuple):
    word, tag = pos_tuple
    if tag.startswith(("JJ", "RB", "VB")):
        return True
    # return True
lyric_feelings_segmentation = [word for word, tag in filter(
    feelings,
    lyric_rm_stopwords_pos
)]

def pronouns(pos_tuple):
    word, tag = pos_tuple
    if tag.startswith(("PRP")):
        return True
    elif word == "i":
        return True
lyric_pronouns_segmentation = [word for word, tag in filter(
    pronouns,
    lyric_pos
)]

def nouns(pos_tuple):
    word, tag = pos_tuple
    if tag.startswith(("NN")):
        return True
lyric_nouns_segmentation = [word for word, tag in filter(
    nouns,
    lyric_rm_stopwords_pos
)]


# calculate each word's word count
def countOccurrence(a):
  k = {}
  for j in a:
    if j in k:
      k[j] +=1
    else:
      k[j] =1
  return k

# lyric_segmentation_count = countOccurrence(lyric_segmentation)
# lyric_feelings_segmentation_count = countOccurrence(lyric_feelings)
# lyric_pronouns_segmentation_count = countOccurrence(lyric_feelings)
# lyric_segmentation_count_rm_stopwords = countOccurrence(lyric_rm_stopwords_segmentation)

# frequency distributions
frequency_distribution_all = nltk.FreqDist(lyric_segmentation)
frequency_distribution_feelings = nltk.FreqDist(lyric_feelings_segmentation)
frequency_distribution_pronouns = nltk.FreqDist(lyric_pronouns_segmentation)
frequency_distribution_nouns = nltk.FreqDist(lyric_nouns_segmentation)
frequency_distribution_rm_stopwords = nltk.FreqDist(lyric_rm_stopwords_segmentation)

# make into dataframes
frequency_distribution_all_df = pd.DataFrame(list(frequency_distribution_all.most_common())) # the nltk.FreqDist object has built in functions, like taking the most_common will sort the frequency distribution 
frequency_distribution_all_df.columns = ["word", "count"]

frequency_distribution_feelings_df = pd.DataFrame(list(frequency_distribution_feelings.most_common()))
frequency_distribution_feelings_df.columns = ["word", "count"]

frequency_distribution_pronouns_df = pd.DataFrame(list(frequency_distribution_pronouns.most_common()))
frequency_distribution_pronouns_df.columns = ["word", "count"]

frequency_distribution_nouns_df = pd.DataFrame(list(frequency_distribution_nouns.most_common()))
frequency_distribution_nouns_df.columns = ["word", "count"]

frequency_distribution_rm_stopwords_df = pd.DataFrame(list(frequency_distribution_rm_stopwords.most_common()))
frequency_distribution_rm_stopwords_df.columns = ["word", "count"]

# output each category into csv
frequency_distribution_all_df.to_csv (abs_path + "//" + file_name + "_all_word_count.csv", index = False, header=True, encoding="UTF-8")
frequency_distribution_feelings_df.to_csv (abs_path + "//" + file_name + "_feelings_word_count.csv", index = False, header=True, encoding="UTF-8")
frequency_distribution_pronouns_df.to_csv (abs_path + "//" + file_name + "_pronouns_word_count.csv", index = False, header=True, encoding="UTF-8")
frequency_distribution_nouns_df.to_csv (abs_path + "//" + file_name + "_nouns_word_count.csv", index = False, header=True, encoding="UTF-8")
frequency_distribution_rm_stopwords_df.to_csv (abs_path + "//" + file_name + "_rm_stopwords_word_count.csv", index = False, header=True, encoding="UTF-8")

# compile into an excel file
def excel_export():
    global tab
    tab = "compiled"
    all_files = glob.glob(os.path.join(abs_path + "//", (file_name + "*.csv"))) # Find all files that have a .csv extenstion and certain prefix
    all_files.sort(key=os.path.getctime)

    writer = pd.ExcelWriter(os.path.join(abs_path + "//" + file_name + "_" + tab + ".xlsx"), engine='xlsxwriter')

    for f in all_files:
        df = pd.read_csv(f)
        df.to_excel(writer, sheet_name=os.path.splitext(os.path.basename(f))[0][20:], index=False)
    writer.save()
    
excel_export()

# Manual frequency distribution count and sorting method
# # sort word count
# sorted_keys = sorted(lyric_segmentation_count, key=lyric_segmentation_count.get, reverse=True)
# sorted_lyric_segmentation_count = {}
# for w in sorted_keys:
#     sorted_lyric_segmentation_count[w] = lyric_segmentation_count[w]

# # sort word count
# sorted_keys_rm_stopwords = sorted(lyric_segmentation_count_rm_stopwords, key=lyric_segmentation_count_rm_stopwords.get, reverse=True)
# sorted_lyric_segmentation_rm_stopwords_count = {}
# for w in sorted_keys_rm_stopwords:
#     sorted_lyric_segmentation_rm_stopwords_count[w] = lyric_segmentation_count_rm_stopwords[w]

# # convert dictionary to dataframe
# sorted_lyric_segmentation_count_df = pd.DataFrame(list(sorted_lyric_segmentation_count.items()))
# sorted_lyric_segmentation_count_df = sorted_lyric_segmentation_count_df.rename(columns={0: "word", 1: "count"})
# frequency_distribution_all_df = pd.DataFrame(list(frequency_distribution_all.most_common()))
# frequency_distribution_df = frequency_distribution_df.rename(columns={0: "word", 1: "count"})
# sorted_lyric_segmentation_count_df.to_csv (abs_path + "//" + file_name + "_word_count.csv", index = False, header=True, encoding="UTF-8")